In [299]:
import openpyxl
from openpyxl import load_workbook
import numpy as np
import pandas as pd

In [300]:
pathExcel = r"C:\Users\Mathe\Downloads\Calculo Fluxo Teste\Calculo_Fluxo (1).xlsx"

In [301]:
wb = load_workbook(filename=pathExcel, 
                   read_only=True)
try:
    ws = wb['PRICE_CORR']
except:
    ws = wb['SAC_CORR']

# Read the cell values into a list of lists
data_rows = []
for row in ws['A18':'A500']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)


In [302]:

# Transform into dataframe
df = pd.DataFrame(data_rows)
df.replace('',np.nan, inplace=True)
df = df.dropna()
lastRow = len(df) + 17
df.columns = ['numParcela']
try:
    df['numParcela'] = df['numParcela'].replace("I", 0)
except:
    print('erro')
df['numParcela'] = df['numParcela'].astype(int)

In [303]:
#Append Data Vencimento
date_rows = []
for row in ws['B18':f'B{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)

df["data_vencto"] = pd.DataFrame(date_rows)

In [304]:
#Append Valor Amortização
date_rows = []
for row in ws['C18':f'C{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)
df["amortizacao"] = pd.DataFrame(date_rows)

In [305]:
#Append Juros
date_rows = []
for row in ws['D18':f'D{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)
df["juros"] = pd.DataFrame(date_rows)

In [306]:
numCCI = "123456"

In [307]:
df['amortizacao'] = df['amortizacao'].replace(np.nan, 0)
df['juros'] = df['juros'].replace(np.nan, 0)
df['valor_principal'] = round(df['amortizacao'],2)
df['valor_futuro'] = round(df['amortizacao']+df['juros'],2)

#Valor CCI
df['codigo_integracao'] = numCCI

#Valores Padrões
df['id_tipo_parcela'] = "2"
df['Aplica_Correcao'] = "S"
df['Aplica_Juros'] = "S"
df['periodicidade'] = "1"

In [308]:
df = df[['codigo_integracao', 'data_vencto', 'id_tipo_parcela', 'numParcela', 'valor_principal', 'valor_futuro', 'Aplica_Correcao', 'Aplica_Juros', 'periodicidade']]
df['data_vencto'] = pd.to_datetime(df['data_vencto'], format='%d/%m/%y').dt.strftime('%d/%m/%Y')

In [309]:
df_remove = df.loc[(df['valor_principal'] == 0)]
df = df.drop(df_remove.index)

In [310]:
if df['numParcela'].iloc[0] == 2:
    df['numParcela'] = df['numParcela'] - 1

df['numParcela'] = np.where(df['valor_principal'] >0, df['numParcela'].apply(str)+"/"+str(df['numParcela'].iloc[-1]), 0)
df = df.rename(columns={'numParcela': 'numero_parcela'})
df

,codigo_integracao,data_vencto,id_tipo_parcela,numero_parcela,valor_principal,valor_futuro,Aplica_Correcao,Aplica_Juros,periodicidade
2,123456,10/01/2023,2,1/336,162.66,2380.59,S,S,1
3,123456,10/02/2023,2,2/336,163.97,2380.59,S,S,1
4,123456,10/03/2023,2,3/336,165.28,2380.59,S,S,1
5,123456,10/04/2023,2,4/336,166.61,2380.59,S,S,1
6,123456,10/05/2023,2,5/336,167.94,2380.59,S,S,1
...,...,...,...,...,...,...,...,...,...
333,123456,10/08/2050,2,332/336,2287.40,2380.59,S,S,1
334,123456,10/09/2050,2,333/336,2305.75,2380.59,S,S,1
335,123456,10/10/2050,2,334/336,2324.23,2380.59,S,S,1
336,123456,10/11/2050,2,335/336,2342.87,2380.59,S,S,1


In [105]:
import pprint
jsonTeste = df.to_json(orient='records').replace("\\/", str('/'))
#pprint.pprint(jsonTeste)

In [106]:
#Valores Cálculo Fluxo
dict_values = {'valorBrutoCF':"", 'tabelaCF': '', 'IOFCF':'', 'TACCF': '', 'valorRegistroCF': '', 'valorLiquidoCF': '', 'prazoMesesCF': '', 'taxaMesCF': '', 'primeiraParcelaCF':'',
                'valorImovelCF': '', 'carenciaCF': '', 'mesPularParcelaCF':'', 'CETCF':'', 'prazoContratoCF': ''
                }

#Transformar KEYS em lista
listKey =  list(dict_values.keys())
i = 0 

for row in ws['C2':'C15']:
    for cell in row:
        #Definição de valor da chave
        keyValue = listKey[i]
        i+=1

        valorCelula = cell.value

        if type(valorCelula) == float:
            valorCelula = round(valorCelula, 2)

        dict_values[keyValue] = valorCelula



In [107]:
dict_values

{'valorBrutoCF': 205206,
 'tabelaCF': 'SAC',
 'IOFCF': 0,
 'TACCF': 6300.0,
 'valorRegistroCF': 18906,
 'valorLiquidoCF': 180000,
 'prazoMesesCF': 360,
 'taxaMesCF': 0.01,
 'primeiraParcelaCF': 2355.36,
 'valorImovelCF': 390000,
 'carenciaCF': 1,
 'mesPularParcelaCF': 0,
 'CETCF': 0.13,
 'prazoContratoCF': 360}